# Getting Started with Llamaindex

Today we are going to create a simple RAG (retrieval-augmented-generation) question-answering chatbot and deploy it.

This represents a second way to easily build and deploy RAG projects. We covered the first way in week 8 - using RAGatouille, Gradio, and Huggingface spaces.

## Prerequisites

- Make sure you have a recent version of node (I'm using node v18)
- Make sure you are using python 3.11
  - use mise (or asdf or pyenv) to set the python version

## Create a Llamaindex project
- `npx create-llama@latest`
  - Project Name? test-llama
  - Template? Chat
  - Framework? FastAPI (Python)
  - Generate a NextJS frontend? Yes
  - Obervability? No
  - OpenAI key?
    - you can use mine if you want to take this approach - ask me
  - Data source? Example PDF
  - Another data source? No
  - LlamaParse? No
  - Vector database? ChromaDB
  - Agent? leave blank and just press enter
  - Proceed? Just generate code

## Start the backend
- `cd test-llama/backend`
- `poetry install`
- `poetry shell`
- edit .env: uncomment CHROMA_PATH and set it to `chroma`
- `poetry run generate`
- `python main.py`
- read backend/README.md for more information

### Do you get an error about dotenv not being found?

If you are using mise and you get this error when you run python main.py, let me know. 
The problem is that mise put its python path in front of poetry's python path in $PATH.

I had to uninstall mise and then re-install it, adding the mise shims to my path instead of activating mise.

## Start the frontend
- open a new terminal window
- `cd ../frontend`
- `npm install`
- `npm run dev`
- visit http://localhost:3000 and ask "what is the format of a postcard?"

Congratulations! You just launched your first Llamaindex app. 

Now let's push it to github and deploy it so you can share it with your mother.

## Edit Dockerfile

Replace the last line of the Dockerfile in the frontend directory with the following in order to make it work with Render.

`CMD ["npx", "-y", "serve@latest", "out"]`

Docker is amazing. It's a way to package your application along with its external (operating system) dependencies like the exact versions of python, node, and sqlite. This gives you complete control over the environment in which your application runs. It's like having another operating system inside your operating system.

## Push to github

- Create a new github repo called test-llama. Make it public but don't add a readme.
- From the project root directory
  - `git add .`
  - `git commit -m 'initial commit'`
  - `git remote add origin https://github.com/[your user name]/test-llama.git`
  - `git push --set-upstream origin main`

## Create a free account on Render to deploy your project

*Previous to Render, I tried deploying the project to Vercel and Microsoft Azure. Vercel only supports python 3.9 (they say they support 3.12 but it's broken) and the demo project we just created requires python 3.11. Azure kept giving me the error "The subscription is not allowed to create or update the serverfarm", which appears to be a common error over the past several months - you're supposed to submit a support ticket to ask them to fix it, but submitting a support ticket requires a paid subscription.*

- Create a Render account
  - go to https://render.com click Get Started for Free and create an account using your Github credentials.

## Deploy the backend

We will deploy the backend using Docker.

- from the dashboard select Create a new Web Service
- select Build and Deploy from a Git repository
- connect it to your test-llama github repository
- name it [your name]-test-llama-backend
- set the root directory to backend
- set the runtime to Docker
- change the instance type to Free
- add the following environment variables from .env
  - MODEL_PROVIDER=openai
  - MODEL=gpt-3.5-turbo
  - EMBEDDING_MODEL=text-embedding-3-large
  - EMBEDDING_DIM=1024
  - OPENAI_API_KEY=your OpenAI key
  - TOP_K=3
  - CHROMA_PATH=chroma
  - FILESERVER_URL_PREFIX=https://[your name]-test-llama-backend/api/files
  - APP_HOST=0.0.0.0
  - APP_PORT=80
  - ENVIRONMENT=prod
  - SYSTEM_PROMPT=You are a helpful assistant who helps users with their questions.
- click Create Web Service

## Deploy the frontend

We will also deploy the frontend using Docker.

- from the dashboard create a new web service
- select Build and Deploy from a Git repository
- connect it to your test-llama github repository
- name it [your name]-test-llama-frontend
- set the root directory to frontend
- set the runtime to Docker
- set the instance type to Free
- add the following environment variable
  - NEXT_PUBLIC_CHAT_API=https://[your name]-test-llama-backend.onrender.com/api/chat
- click Create Web Service

## Test your new service

Go to https://[your-name]-test-llama-frontend.onrender.com and give it a try!

## Where to go from here?

What if you wanted to search your own files? Take a look at the backend/data directory. It appears that you can simply replace 101.pdf with your own files, then re-run `poetry run generate` to update the Chroma database with your new data.

We will learn more about Llamaindex over the next several months.